[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/bobleesj/quantem.widget/blob/main/notebooks/show1d/show1d_simple.ipynb)

# Show1D — Quick Demo

Interactive 1D viewer for spectra, line profiles, and time series.

In [1]:
try:
    import google.colab
    !pip install -q -i https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple/ quantem-widget
except ImportError:
    pass  # Not in Colab, skip

In [2]:
try:
    %load_ext autoreload
    %autoreload 2
    %env ANYWIDGET_HMR=1
except Exception:
    pass  # autoreload unavailable (Colab Python 3.12+)

env: ANYWIDGET_HMR=1


In [3]:
import numpy as np
from quantem.widget import Show1D


def make_eels_spectrum(n=512, seed=0):
    """Simulate EELS spectrum with zero-loss peak, plasmon, and core-loss edges."""
    rng = np.random.default_rng(seed)
    energy = np.linspace(-20, 800, n)  # eV
    # Zero-loss peak
    zlp = 1000 * np.exp(-0.5 * (energy / 3) ** 2)
    # Plasmon peaks
    plasmon = 200 * np.exp(-0.5 * ((energy - 15) / 5) ** 2)
    plasmon += 80 * np.exp(-0.5 * ((energy - 30) / 8) ** 2)
    # Power-law background
    bg = np.where(energy > 5, 5000 * (energy.clip(5, None) / 5) ** -2.5, 0)
    # Core-loss edge (e.g. O-K at 532 eV)
    edge_onset = 532
    edge = np.where(energy > edge_onset, 15 * ((energy - edge_onset).clip(0, None) / 50) ** 0.4 * np.exp(-(energy - edge_onset) / 200), 0)
    spec = zlp + plasmon + bg + edge + rng.poisson(2, n).astype(np.float32)
    return energy.astype(np.float32), spec.astype(np.float32)


def make_convergence_curve(n=200, seed=0):
    """Simulate optimization convergence (e.g. ptychography reconstruction)."""
    rng = np.random.default_rng(seed)
    epochs = np.arange(n, dtype=np.float32)
    loss = 10.0 * np.exp(-0.02 * epochs) + 0.1 + 0.05 * rng.standard_normal(n)
    return epochs, loss.astype(np.float32)

print("Generator functions ready.")

Generator functions ready.


/Users/macbook/miniforge3/envs/bob-env/lib/python3.11/site-packages/anywidget/_util.py:283: UserWarning: anywidget: Live-reloading feature is disabled. To enable, please install the 'watchfiles' package.
  start_thread=_should_start_thread(path),


## Single EELS Spectrum

In [4]:
energy, spec = make_eels_spectrum()
Show1D(spec, x=energy, title="EELS Spectrum", x_label="Energy Loss", x_unit="eV", y_label="Counts")

Show1D(512 points)

## Multiple Spectra Overlay

Compare spectra from different sample regions.

In [5]:
energy, spec1 = make_eels_spectrum(seed=0)
_, spec2 = make_eels_spectrum(seed=1)
_, spec3 = make_eels_spectrum(seed=2)

Show1D(
    [spec1, spec2, spec3],
    x=energy,
    labels=["Region A", "Region B", "Region C"],
    title="EELS — Multi-region Comparison",
    x_label="Energy Loss",
    x_unit="eV",
    y_label="Counts",
)

Show1D(3 traces × 512 points)

## Optimization Convergence

Track reconstruction error over iterations.

In [6]:
epochs, loss = make_convergence_curve()
Show1D(loss, x=epochs, title="Ptychography Convergence", x_label="Iteration", y_label="Error", log_scale=True)

Show1D(200 points)

## State Inspection

In [7]:
w = Show1D(spec, x=energy, title="EELS Spectrum", x_label="Energy Loss", x_unit="eV")
w.summary()

EELS Spectrum
════════════════════════════════
Traces:   1 × 512 points
Labels:   Trace
X range:  -20 – 800
X axis:   Energy Loss (eV)
  [0] Trace: mean=40.28  min=7.794e-06  max=3850  std=225.8
Display:  linear | grid
